<a href="https://colab.research.google.com/github/SeonHyungJo/tensorflow-2.0-study/blob/master/Keras%20Docs/%EC%BC%80%EB%9D%BC%EC%8A%A4_%ED%95%A8%EC%88%98%ED%98%95_API_%EC%B2%AB%EA%B1%B8%EC%9D%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 케라스 함수형 API 첫걸음

# 첫 예시: 밀집 연결 네트워크

밀집 연결 네트워크를 구현하기에는 Sequential 모델이 더 적합한 선택이겠지만, 아주 간단한 예시를 위해서 케라스 함수형 API로 구현해 보겠습니다.

레이어 인스턴스는 (텐서에 대해) 호출 가능하고, 텐서를 반환합니다
인풋 텐서와 아웃풋 텐서는 Model을 정의하는데 사용됩니다.
이러한 모델은 케라스 Sequential 모델과 완전히 동일한 방식으로 학습됩니다.

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

# 이는 텐서를 반환합니다
inputs = Input(shape=(784,))

# 레이어 인스턴스는 텐서에 대해 호출 가능하고, 텐서를 반환합니다
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# 이는 Input 레이어와 3개의 Dense 레이어를
# 포함하는 모델을 만들어 냅니다
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels)  # starts training

# 레이어처럼, 모든 모델이 호출 가능합니다

함수형 API를 사용하면 학습된 모델을 재사용하기 편리합니다: 어느 모델이건 텐서에 대해 호출하여 레이어처럼 사용할 수 있습니다. 모델을 호출하면 모델의 구조만 재사용하는 것이 아니라 가중치까지 재사용되는 것임을 참고하십시오.

In [ ]:
x = Input(shape=(784,))
# 다음이 실행되어 위에서 정의한 10 방향 소프트맥스를 반환합니다.
y = model(x)

이를 사용하면, 예를 들어 인풋의 시퀀스를 처리할 수 있는 모델을 빠르게 만들 수 있습니다. 코드 한 줄로 이미지 분류 모델을 비디오 분류 모델로 바꿀 수 있는 것입니다.

In [ ]:
from keras.layers import TimeDistributed

# 20 시간 단계의 시퀀스에 대한 인풋 텐서로,
# 각각 784 차원의 벡터를 담고 있습니다.
input_sequences = Input(shape=(20, 784))

# 인풋 시퀀스의 모든 시간 단계에 이전 모델을 적용합니다.
# 이전 모델의 아웃풋이 10 방향 소프트맥스였으므로,
# 아래 레이어의 아웃풋은 크기 10의 벡터 20개로 이루어진 시퀀스입니다.
processed_sequences = TimeDistributed(model)(input_sequences)

# 다중-인풋과 다중-아웃풋 모델

다중 인풋과 아웃풋을 다루는 모델은 함수형 API를 사용하기 특히 적합한 사례입니다. 함수형 API를 사용해서 복잡하게 얽힌 많은 수의 데이터 줄기를 간편하게 관리할 수 있습니다.

다음의 모델을 고려해 봅시다. 얼마나 많은 트위터에서 특정 뉴스 헤드라인이 얼마나 낳은 리트윗과 좋아요를 받을지 예측하려고 합니다. 모델에 대한 주요 인풋은 단어 시퀀스로 표현된 헤드라인 자체이지만, 문제를 조금 더 흥미롭게 풀기 위해, 보조 인풋으로 헤드라인이 올려진 시간 등과 같은 추가 데이터를 받도록 합시다. 두 개의 손실 함수를 통해 모델을 감독합니다. 주요 손실 함수를 모델의 초기 단계에 사용하는 것이 심화 모델에 있어 적절한 정규화 메커니즘입니다.

In [1]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

# 헤드라인 인풋: 1에서 10000사이의 100개 정수로 이루어진 시퀀스를 전달받습니다
# "name"인수를 전달하여 레이어를 명명할 수 있음을 참고하십시오.
main_input = Input(shape=(100,), dtype='int32', name='main_input')

# 이 임베딩 레이어는 인풋 시퀀스를
# 밀집 512 차원 벡터의 시퀀스로 인코딩합니다
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)

# 장단기 메모리는 벡터 시퀀스를 전체 시퀀스에 대한
# 정보를 포함하는 단일 벡터로 변형합니다
lstm_out = LSTM(32)(x)

다음에서는 보조 손실을 추가하여, 메인 손실이 높아지더라도 장단기 메모리와 임베딩 레이어가 매끄럽게 학습될 수 있도록 합니다.

In [6]:
auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

이 시점에서 보조 인풋을 장단기 메모리 아웃풋에 연결하여 모델에 전달한다.

In [ ]:
auxiliary_input = Input(shape=(5,), name='aux_input')
x = keras.layers.concatenate([lstm_out, auxiliary_input])

# 심화 밀집 연결 네트워크를 상층에 쌓습니다
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# 끝으로 주요 로지스틱 회귀 레이어를 추가합니다
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

이는 두 개의 인풋과 두 개의 아웃풋을 갖는 모델을 정의한다.

In [ ]:
model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])

이제 모델을 컴파일하고 0.2의 가중치를 보조 손실에 배당합니다. 리스트나 딕셔너리를 사용해서 각기 다른 아웃풋에 별도의 loss_weights 혹은 loss를 특정할 수 있습니다. 다음에서는 loss인수로 단일 손실을 전달하여 모든 아웃풋에 대해 동일한 손실이 사용되도록 합니다.

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
              loss_weights=[1., 0.2])

인풋 배열과 표적 배열의 리스트를 전달하여 모델을 학습시킬 수 있습니다:

In [ ]:
model.fit([headline_data, additional_data], [labels, labels],
          epochs=50, batch_size=32)

인풋과 아웃풋이 명명되었기에 (이름은 "name" 인수를 통해 전달합니다), 다음과 같이 모델을 컴파일 할 수 있습니다:

In [ ]:
model.compile(optimizer='rmsprop',
              loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
              loss_weights={'main_output': 1., 'aux_output': 0.2})

또한 모델을 다음과 같이도 학습시킬 수 있습니다:
model.fit({'main_input': headline_data, 'aux_input': additional_data},
          {'main_output': labels, 'aux_output': labels},
          epochs=50, batch_size=32)

# 공유 레이어

함수형 API의 또 다른 유용한 사용처는 공유 레이어를 사용하는 모델이다.

트윗 데이터셋을 고려해보면. 두 개의 다른 트윗을 두고 동일한 사람이 작성했는지 여뷰를 가려내는 모델을 만들고자 한다.

이를 구현하는 방법의 하나는 두 개의 트윗을 두 벡터로 인코딩하고 그 두 벡터를 연결한 후 로지스틱 회귀를 더하는 모델을 만드는 것입니다; 이는 두 트윗이 동일한 저자를 공유할 확률을 출력합니다. 그에 따라 모델은 양성 트윗 쌍과 음성 트윗 쌍에 대해서 학습됩니다.

문제가 대칭적이므로, 첫 번째 트윗을 인코딩하는 메커니즘을 (가중치 등을 포함해) 전부 재사용하여 두 번째 트윗을 인코딩해야 합니다. 이 예시에서는 공유된 장단기 메모리 레이어를 사용해 트윗을 인코딩 합니다.

함수형 API로 이 모델을 만들어 봅시다. (280, 256) 형태의 이진 행렬, 다시 말해 256 크기의 벡터 280개로 이루어진 시퀀스를 트윗에 대한 인풋으로 받습니다 (여기서 256 차원 벡터의 각 차원은 가장 빈번한 256 부호의 자모 중 해당 부호의 유/무를 인코딩합니다).

In [ ]:
import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model

tweet_a = Input(shape=(280, 256))
tweet_b = Input(shape=(280, 256))

제 각기 다른 인풋에 걸쳐 레이어를 공유하려면, 간단히 레이어를 한 번만 인스턴스화하고 인풋마다 그 레이어를 호출하면 됩니다:

In [ ]:
# 이 레이어는 행렬을 인풋으로 전달받고
# 64 크기의 벡터를 반환합니다
shared_lstm = LSTM(64)

# 동일한 레이어 인스턴스를
# 여러 번 재사용하는 경우, 레이어의
# 가중치 또한 재사용됩니다
# (그렇기에 이는 실질적으로 *동일한* 레이어입니다)
encoded_a = shared_lstm(tweet_a)
encoded_b = shared_lstm(tweet_b)

# 이제 두 벡터를 연결할 수 있습니다:
merged_vector = keras.layers.concatenate([encoded_a, encoded_b], axis=-1)

# 그리고 로지스틱 회귀를 상층에 추가합니다
predictions = Dense(1, activation='sigmoid')(merged_vector)

# 트윗 인풋을 예측에 연결하는
# 학습 가능한 모델을 정의합니다
model = Model(inputs=[tweet_a, tweet_b], outputs=predictions)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit([data_a, data_b], labels, epochs=10)

잠시 멈추고 공유 레이어의 아웃풋 혹은 아웃풋 형태를 어떻게 해석해야 할지 알아봅시다.

# 레이어 "node"의 개념

어느 인풋에 대해서 레이어를 호출하면, 새로운 텐서(레이어의 아웃풋)가 생성됩니다. 또한 "node"가 레이어에 추가되어 인풋 텐서를 아웃풋 텐서에 연결합니다. 동일한 레이어를 여러 번 호출하면, 레이어는 0, 1, 2…로 색인이 달린 여러 개의 노드를 소유하게 됩니다.

케라스의 이전 버전에서는, layer.get_output()를 통해 레이어 인스턴스의 아웃풋 텐서를 얻거나, 혹은 layer.output_shape를 통해 아웃풋 형태를 얻을 수 있었습니다. 이러한 방식이 여전히 가능하기는 하지만 (output으로 대체된 get_output()은 제외), 레이어가 여러 인풋에 연결된 경우는 어떻게 하시겠습니까?

하나의 레이어가 하나의 인풋에만 연결되는 한, .output은 혼동없이 레이어의 단일 아웃풋을 반환할 것입니다:

In [ ]:
a = Input(shape=(280, 256))

lstm = LSTM(32)
encoded_a = lstm(a)

assert lstm.output == encoded_a

In [8]:
a = Input(shape=(280, 256))
b = Input(shape=(280, 256))

lstm = LSTM(32)
encoded_a = lstm(a)
encoded_b = lstm(b)

lstm.output

<tf.Tensor 'lstm_2/PartitionedCall:0' shape=(None, 32) dtype=float32>

In [ ]:
assert lstm.get_output_at(0) == encoded_a
assert lstm.get_output_at(1) == encoded_b

input_shape와 output_shape의 경우도 마찬가지 입니다: 레이어가 하나의 노드만 보유하는 한, 혹은 모든 노드가 동일한 인풋/아웃풋 형태를 갖는 한, "layer output/input shape"의 개념이 명확히 정의되며 layer.output_shape/layer.input_shape은 동일한 형태를 반환합니다. 하지만 만약 동일한 Conv2D 레이어를 (32, 32, 3) 형태의 인풋에 적용한 후 (64, 64, 3) 형태의 인풋에도 적용했다면, 레이어는 여러 종류의 인풋/아웃풋 형태를 보유하게 됩니다. 이러한 경우 각 인풋/아웃풋 형태가 속한 노드의 색인을 특정해서 불러와야 합니다:

In [ ]:
a = Input(shape=(32, 32, 3))
b = Input(shape=(64, 64, 3))

conv = Conv2D(16, (3, 3), padding='same')
conved_a = conv(a)

# 지금까진 하나의 인풋만 존재하므로 다음은 제대로 동작합니다:
assert conv.input_shape == (None, 32, 32, 3)

conved_b = conv(b)
# 이제 `.input_shape`은 제대로 작동하지 않지만, 다음은 동작합니다:
assert conv.get_input_shape_at(0) == (None, 32, 32, 3)
assert conv.get_input_shape_at(1) == (None, 64, 64, 3)